In [1]:
# Import libraries
import pandas as pd
import json
import tweepy
import psycopg2
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, plot
import time
init_notebook_mode(connected=True)

In [2]:
# Keys
consumer_key = 'Eb7Q832xETPncAvbICrimtbgQ'
consumer_secret = 'myJ84Ry5Zete9zgqJtQb7pZK8fw9oAqF7CssGQAj0PXVdH5pP0'
access_token = '2760228716-kr6JayzHlTNbEf0JGmhaGdJzsVhXWqRU83nolpK'
access_secret = 'h7rgFGE0IrfQQXxotXuvhfAhZJq08aD7em6V5fo1fEO22'
plotly_username = 'gunajit23'
plotly_key = 'ULuUce6nAp56xGWMIH2W'
stream_id = 'q3i23ommyd'

In [3]:
#Set plotly credentials
plotly.tools.set_credentials_file(username=plotly_username, api_key=plotly_key)

In [4]:
# Instance of stream id object for plotting
stream_1 = go.scatter.Stream(token=stream_id,maxpoints=80)

In [5]:
#Setup tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [6]:
# Connect to database
conn_new = psycopg2.connect(dbname='conn_new2', user='postgres')
cursor = conn_new.cursor()

In [7]:
# Create table
cursor.execute("CREATE TABLE IF NOT EXISTS tweets (id SERIAL PRIMARY KEY, user_name VARCHAR, media BOOL, retweetcount INT, favcount INT, created_at TIMESTAMP, tweet text NOT NULL, location VARCHAR);")
conn_new.commit()

In [8]:
# For accessing database
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres@localhost:5432/conn_new2')

In [9]:
import plotly.offline as offline

In [10]:
# Override on_status method of StreamListener 
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        # Filter retweets
        if 'RT @' in status.text or status.retweeted:
            return
        # Checks if tweet contains media
        if 'media' in json.dumps(status._json):
            media = True   
        else:
            media = False
        
        # Insert tweet data to database
        try:
            cursor.execute("INSERT INTO tweets (user_name, media, retweetcount, favcount, created_at, tweet, location) VALUES (%s, %s, %s, %s, %s, %s, %s);", (status.user.screen_name, media, status.retweet_count, status.favorite_count, status.created_at, status.text, status.user.location))
            conn_new.commit()
        except Exception as e:
            print(e)
        
        # Read database
        df = pd.read_sql_query('select * FROM tweets',conn_new)
       
        # Timestamp till minute value
        till_min = pd.to_datetime(df.created_at.dt.strftime('%Y-%m-%d-%H:%M'))
        
        # Tweets plot
        data = Data([ Scatter(x = df.groupby(till_min).size().index, y = df.groupby(till_min).size().values)])
        offline.plot(data, filename='tweets.html')
        
        # Media_tweets plot
        data = Data([ Scatter(x = df[df.media == True].groupby(till_min).size().index, y = df[df.media == True].groupby(till_min).size().values)])
        offline.plot(data, filename='media_tweets.html')
        time.sleep(60)
        
    def on_error(self, status_code):
        if status_code == 420:
            return False    

In [11]:
# Stream tweets after creating a tweepy Stream instance
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=['#nba'])

C:\ProgramData\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




KeyboardInterrupt: 